<h1>
    <br style ="font-family:UCL-SoM-Outline;color:#EA7600"> GROUP H COURSEWORK </br> 
    </p>
</h1>

<div class="image">

<img src="./figure/som_ft.png" width="70%"  align="right">
<h4>
          <p style="font-size:18pt">MSIN0097 Predictive Analytics</p>
          <p style="font-size:18pt;font-family:UCL-SoM-Solid; color:#EA7600;">Student Name: <u>Thomas Zenkner, Ioana Buzelan, Bianca Ursulescu, Jiarui Xin, Christoph Loos</u> </p> 
          <p style="font-size:18pt;font-family:UCL-SoM-Solid; color:#EA7600;">ID: <u>xxx</u> </p> 
          <p style="font-size:18pt;font-family:UCL-SoM-Solid; color:#EA7600;">Email: <u>xxxx</u> </p> 

</h4>

</div>

<br>
<br>
<br>


# COURSEWORK: WARNER MUSIC



### PREDICTING THE SUCCESS OF ARTISTS ON SPOTIFY

# Table of Contents
* [1. Set-up](#1.Set-up)
    * [1.1. Unique Artist Names, etc, etc,](#1.1.-Unique-Artist-Names,-etc,-etc,)
    * [1.2. Connection to **`Spotify API`**](#1.1.-Connection-to-Spotify-API)
* [2. Feature Engineering](#2.-Feature-Engineering)
    * [2.1 Artist Features](#2.1-Artist-Features)
    * [2.2 Playlist Features](#2.2-Playlist-Features)
    * [2.3 User Features](#2.3-User-Features)
    * [2.4 Merge the features into the analytics-ready DataFrame](#2.4-Merge-the-features-into-the-analytics-ready-DataFrame)



## 1. Set-up




In [1]:
# Import all required libraries
import random 
import pandas as pd
import numpy as np
import seaborn as sns
!pip install missingno
import missingno as msno

# To make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
!pip install matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# # Import custom functions from library, named 'spotfunc'
# import spotfunc as spotfunc_v2

# Ignore useless warnings 
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Various other things needed for data preparation
from sklearn.preprocessing import scale
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from scipy.stats import norm

In [2]:
%%time
# Read in sampled data
data = pd.read_csv('cleaned_data.csv', low_memory=False)
print('rows data:',len(data))

# Keep a copy of original data in case of changes made to dataframe
all_artists = data.copy()

# Load playlist data
playlist_ids_and_titles = pd.read_csv('playlists_ids_and_titles.csv',encoding = 'latin-1',error_bad_lines=False,warn_bad_lines=False)

# Drop duplicates
playlist_mapper = playlist_ids_and_titles.drop_duplicates(['id'])

rows data: 3805499
Wall time: 1min 10s


In [ ]:

#DATA CLEANING 
#make all artists with lower case
data['artist_name'] = data['artist_name'].astype(str).str.lower()
data['track_name'] = data['track_name'].astype(str).str.lower()

# As age is more intuitive to interpret than birthyear, we convert 
data["user_age"] = 2019-data["birth_year"]
data['user_age'] = data['user_age'].apply(lambda x: x if x <= 100 else 100) #Ioana: anything greater than 100 will take the value of 100
# Maybe instead of making <100 to 100 maybe make them the median or mean or mode?

data.drop_duplicates()

In [ ]:
# Df contianing only the 4 popular playlists and all their associated IDs
filtered_mapper = playlist_mapper[playlist_mapper["name"].isin(["Hot Hits UK", "Massive Dance Hits", "The Indie List", "New Music Friday"])]


## 2. Exploratory Data Analysis

In [ ]:
data.head()

In [ ]:
# First we look at what data type Pandas attributed to each cloumn
# we need two options to force Pandas to count non-null instances in each coloumn
data.info(verbose=True, null_counts=True)

In [ ]:
data.describe()

<div>
    <b> Initial Overview of Data Load </b>

<p>The first thing we observe is that the majority (28/45) of columns are non-numeric (neither int nor float Dtypes), and got classified as an object instead. While an object could, in theory, hold different values, we know that these objects hold text values due to the fact that they came from a .csv file (which can only hold text or numbers).
    
With that out of the way, we see that not all columns contain 3805499 non-null values (e.g. postal_code only has 2453318 and stream_source_uri only 1043871). This information will be important later on when we prepare the dataset for various ML tasks - we will need to deal with these Null values somehow. We also see that 4 columns (e.g. referral_code or stream_cached) have 0 values in them, which means that we will drop these columns, as they serve no purpose nor benefit.</p> 


In [ ]:
# Chris
# Because vizualising missing data is not possible due to data size
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    if pct_missing != 0:
        print('{} - {}%'.format(col, round(pct_missing*100))) # look into financial product

In [ ]:
# Dropping redundant columns
data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace = True) # not adding any value
data.drop(['referral_code' , 'offline_timestamp', 'stream_cached', 'source'], axis=1, inplace = True) # null columns

<div class="alert alert-success">
<b> Counting categorical values </b>

<p>From counting and analysing the categorical values, we get the information that the location of users follows the population distribution of the UK's biggest cities (which is to be expected). We have circa 150.000 more streams from female customers opposed to males and know that most users use either Vodafone or Boku as their mobile broadband provider. The user_product_type contains a majority of paid users and most streams stem from mobile devices, more precisely mobile devices running iOS. Most Columns do not grant any meaningful insight, but they may still be of great use for our ML model later.</p> 

</div>

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
rs = np.random.RandomState(10)

f, axes = plt.subplots(2, 2, figsize=(15, 7))
sns.despine(left=True)

sns.distplot(data['user_age'], bins=50, fit=norm, ax=axes[0, 0])
sns.distplot(data["stream_length"], bins=50,fit=norm, ax=axes[0, 1])
sns.distplot(data["hour"], bins=24,fit=norm, ax=axes[1, 0])
sns.distplot(data["weekday"], bins=7, fit=norm, ax=axes[1, 1])

axes[0,0].set_xlim(10, 100)
axes[0,1].set_xlim(0,500)

axes[0,0].set_title("Age Distribution")
axes[0,1].set_title("Stream Lenght Distribution")
axes[1,0].set_title("Hour Distribution")
axes[1,1].set_title("Weekday Distribution")

axes[0,0].set_xlabel('')
axes[0,1].set_xlabel('')
axes[1,0].set_xlabel('')
axes[1,1].set_xlabel('')

plt.setp(axes, yticks=[])
plt.tight_layout()

<div >
<b> Age distribution </b>

<p>We see that the age distribution is very much right skewed in comparison to the normal distribution displayed, and has its mode around 23, which means that a large part of user is younger than the average of 28,9 years old. We also see users' age spread out all the way back until 70+ (we capped the age to 100 for this graph, datapoints above 100 are most likely artifacts and/or faulty data. We will deal with these later).
Such a distribution was expected, as young users have much more rapidly adopted to new means of listening to music.</p> 

</div>

<div>
<b> Stream lenght distribution </b>

<p>Most streams are between 150 and 250 seconds long, with a few beeing noticeable longer (up to 800, truncated in this graph for readability purposes) and some more beeing noticeable shorter. The length starts at 30, as spotify does not count streaming for less than 30 seconds as a stream. The mode of 200 is to be expected, as the average song is about 3-4 minutes long, which is 180-240 seconds. </p> 

</div>

<div>
<b>Usage time distribution </b>
 

<p>We can observe with the above three graphs that most users listen between the hours of 15:00 and 20:00. Further, we see a pattern that indicates that Mondays (day=0) and Saturdays (day=5) have the most streams, and the months of June, July and August also have the most streams.</p> 

</div>

In [ ]:
print(plt.style.available)

In [ ]:
mpl.style.use('seaborn-white')

labels = 'Male', 'Female'
sizes = [sum(data["gender"]!="female"), sum(data["gender"]=="female")]
colors = ["#1aa64b","#363837"]

fig = plt.figure(figsize=(8,5))
plt.title('Gender Distribution')
plt.pie(sizes,  explode=(0.04,0), labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, pctdistance=1.2,labeldistance=1.4)

plt.axis('equal')
plt.show()

In [ ]:
# C: Implement threshold for all grpahs? 5 % cutoff seems reasonable

f, axes = plt.subplots(4, 2, figsize=(20, 35))

prob = data.stream_os.value_counts(normalize=True)
threshold = 0.05
mask = prob > threshold
tail_prob = prob.loc[~mask].sum()
prob = prob.loc[mask]
prob['other'] = tail_prob
prob.plot(kind='bar', color=["#1aa64b","#363837"], ax=axes[0, 0])
axes[0,0].set_title("Operating System Distribution")
axes[0,0].set_xticklabels(prob.index, rotation=0)

data_plot_1 = data.stream_device.value_counts(normalize=True)
data_plot_1.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[0,1])
axes[0,1].set_title("Device Distribution")
axes[0,1].set_xticklabels(data_plot_1.index, rotation=0)

data_plot_2 = data.mobile.value_counts(normalize=True)
data_plot_2.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[1,0])
axes[1,0].set_title("Mobile Distribution")
axes[1,0].set_xticklabels(data_plot_2.index, rotation=0)

data_plot_3 = data.partner_name.value_counts(normalize=True)
data_plot_3.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[1,1])
axes[1,1].set_title("Provider Distribution")
axes[1,1].set_xticklabels(data_plot_3.index, rotation=0)

data_plot_4 = data.user_product_type.value_counts()
data_plot_4.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[2,0])
axes[2,0].set_title("User Product Type distribution")
axes[2,0].set_xticklabels(data_plot_4.index, rotation=0)

data_plot_5 = data.stream_source.value_counts(normalize=True)
data_plot_5.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[2,1])
axes[2,1].set_title("Stream Source Distribution")
axes[2,1].set_xticklabels(data_plot_5.index, rotation=0)

data_plot_6 = data.financial_product.value_counts(normalize=True)
data_plot_6.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[3,0])
axes[3,0].set_title("Product Distribution")
axes[3,0].set_xticklabels(data_plot_6.index, rotation=45)

data_plot_7 = data.access.value_counts()
data_plot_7.plot(kind="bar", color=["#1aa64b","#363837"], ax=axes[3,1])
axes[3,1].set_title("User Subscription Type Distribution")
axes[3,1].set_xticklabels(data_plot_7.index, rotation=0)

#Do we want this? 
# data.region_code.value_counts()[:5].plot(kind="bar", color=["C0","C1"], ax=axes[4,0])
# axes[4,0].set_title("User Location Distribution")

plt.xticks(rotation=0)
plt.show()

<div>
<b>User Demographics</b>

<p>Our userbase contains slightly more females then males. The majority use an iOS mobile device, have either Vodafone or Boku as their mobile broadband provider and belong to the user category "paid". Most streaming is done from collections. Users mostly fall under the financial product category "student" and have a "premium" type subscription. Most of them stream form London. Such subscription profile corresponds to the age profile of Spotify users. This is also potentially an evidence that users are likely to be price sensitive. However, it can also be seen that there is still space to increase revenue. </p> 

</div>

In [ ]:
# C: Better placement for graphs / alignment needed - get rid of the empty box

# Next up, we will show how often various playlists come up in the data
# Because we have so many playlists, we only display those who make up more than 2 % of the total data
# https://stackoverflow.com/questions/37598665/how-to-plot-a-value-counts-in-pandas-that-has-a-huge-number-of-different-counts

f, axes = plt.subplots(2, 2, figsize=(20, 20))

prob = data.playlist_name.value_counts(normalize=True)
threshold = 0.02
mask = prob > threshold
tail_prob = prob.loc[~mask].sum()
prob = prob.loc[mask]
prob['other'] = tail_prob
prob.plot(kind='bar', color=["#1aa64b","#363837"], ax=axes[0, 0])
axes[0,0].set_title("Playlists Distribution")

# But here we show all artists who make up more than 5 % of total streams - 2 % would lead to an overcrowded graph
prob = data.artist_name.value_counts(normalize=True)
threshold = 0.05
mask = prob > threshold
tail_prob = prob.loc[~mask].sum()
prob = prob.loc[mask]
prob['other'] = tail_prob
prob.plot(kind='bar', color=["#1aa64b","#363837"], ax=axes[0, 1])
axes[0,1].set_title("Artists Distribution")

# And here we show all tracks that make up more than 1 % of total streams
#prob = data.track_name.value_counts(normalize=True)
#threshold = 0.01
#mask = prob > threshold
#tail_prob = prob.loc[~mask].sum()
#prob = prob.loc[mask]
#prob['other'] = tail_prob
#prob.plot(kind='bar', color=["#1aa64b","#363837"], ax=axes[1, 0])
#axes[1,0].set_title("Tracks Distribution")
#plt.show()


<div>
<b>Playlist distribution</b>

<p>The Playlists suggested by the Warner Analysts (Hot Hits UK, Massive Dance Hits, The Indie List, New Music Friday) are not very popular, with the exception of Hot Hits UK, which indeed is the very most popular playlist. The others do not show up at all with the threshold of 2%, however, we see that the majority of playlists are small ones.</p> 

</div>

<div>
<b>Artist distribution</b>

<p>For artists, we see that only 6 artists have more than 5 % of total streams each, with Charlie Puth being the "Most popular artist" by far, followed by Dua Lipa and Lukas Graham.</p> 

</div>

<div>
<b>Track distribution</b>

<p>For tracks, we see that only 13 tracks have more than 2 % of total streams each, with "7 Years" being the "Most popular Track" by far. This is a Lukas Graham track. </p> 

</div>

In [ ]:
# Bianca 
# Now we want to visualize the time series distribution of streams using the 'date' column  

# Create a DataFrame of the total number of streams per date  
df_date = data['date'].value_counts()
df_date=pd.DataFrame(df_date.sort_index())
df_date.reset_index(level=df_date.index.names, inplace=True) 
df_date=df_date.rename(columns={"index": "Date"})
df_date=df_date.rename(columns={"date":"Number Of Streams"}) 

# Create the bar graph to better visualize time series
sns.set(style="whitegrid")
f, ax = plt.subplots(figsize=(6, 15))
sns.barplot(x=(df_date['Number Of Streams']), y=df_date['Date'],
            data=df_date,            
            label="Number of streams by month",
            orient = "h")

<div>
<b> Time series distribution of streams </b>

<p> We notice that the number of streams follows an upward trend. This indicates that Spotify, as expected, has raised in demand from year to year, seeing a significant increase in the last few years. However, there are some declines at certain dates, which shows that there might be a  degree of seasonality in the time series.</p> 

</div>

<h1>
    <u>Feature Engineering</u> 
</h1>

## Adding The Dependent Variable

To facilitate data exploration, the successful artists will be identified at this stage, in order to understand their users' behaviours better.

There is more than one way of adding the dependent variable. The following will use two methods. Two columns will be created: 
- "successful" - taking value of 1 for the rows containing artist name AND one of the 4 top playlist, otherwise 0
- "success" - taking value of 1 for the rows containing artist name who showed up on one of the 4 top playlist at least once in the entire dataset, otherwise 0

In [ ]:
# select relevant playlists 
successid=filtered_mapper["id"].unique().tolist()

# Define variable which looks at "if artist && top playlist ==> successful, otherwise unsuccessful"
success = data["playlist_id"].isin(successid)
data["successful"] = success

#============second variable==============
# Functon that retrieves a list with all successful artists (who showed up on a top playlist at least once)
def get_successful_artists(all_artists):
    records_successful_playlists = all_artists[all_artists['playlist_id'].isin(successid)]
    return records_successful_playlists['artist_name'].unique()

successful_artists = get_successful_artists(data)

# Define variable which looks at "if successful artist ==> successful, otherwise unsuccessful"
# Create column with dependent variable "success", which takes value of 1if the artist is in the successful list
data['success'] = np.where(data['artist_name'].isin(successful_artists), 1, 0)

In [ ]:
# can have successful artists in both buckets, but only first is successfull artists in successfull playlists
print('successful artists:', data.loc[data['successful'] == 1]['artist_name'].nunique())
print('unsuccessful artists:', data.loc[data['successful'] == 0]['artist_name'].nunique())
data['successful'].value_counts()

In [ ]:
# If artists is successfull, regardless of playlist, its 1
print('successful artists:', data.loc[data['success'] == 1]['artist_name'].nunique())
print('unsuccessful artists:', data.loc[data['success'] == 0]['artist_name'].nunique())
data['success'].value_counts(normalize=True)

In [ ]:
mpl.style.use('seaborn-white')

top_artists_df = data.loc[data['success'] == 1]['artist_name'].value_counts()[:10].sort_values(ascending=True)
objects = top_artists_df.keys()
y_pos = np.arange(len(objects))
performance = top_artists_df.values
plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
plt.yticks(y_pos, objects)
plt.xlabel('Number of Occurrences')
plt.title('Top 10 most streamed successful artists')
plt.show()

In [ ]:
#Successful Artists DF:
successful_artists_df = data.loc[data['artist_name'].isin(successful_artists)]
print('Total successful artists:', successful_artists_df['artist_name'].nunique())

top_unpopular_playlists = successful_artists_df.loc[successful_artists_df['successful']==False]['playlist_name'].value_counts()[:10].sort_values(ascending=True)
objects = top_unpopular_playlists.keys()
y_pos = np.arange(len(objects))
performance = top_unpopular_playlists.values


plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
plt.yticks(y_pos, objects)
plt.xlabel('Number of Occurrences')
plt.title('Top 10 playlists most popular artists show up, different than "The 4"')
plt.show()

In [ ]:
# stronger recommendation based on other playlists - key playlists may not be relevant any more, many other playlists have way more streams
top_unpopular_playlists = successful_artists_df.loc[successful_artists_df['success']==1]['playlist_name'].value_counts()[:10].sort_values(ascending=True)
objects = top_unpopular_playlists.keys()
y_pos = np.arange(len(objects))
performance = top_unpopular_playlists.values

plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
plt.yticks(y_pos, objects)
plt.xlabel('Number of Occurrences')
plt.title('Top Playlists of Successful Artists')
plt.show()


In [ ]:
#"Hot Hits UK", "Massive Dance Hits", "The Indie List", "New Music Friday"
print('From a total of {} successful artists'.format(len(successful_artists)))
print('Total successful artists showing up on Hot Hits UK:', successful_artists_df.loc[successful_artists_df['playlist_name'] == 'Hot Hits UK']['artist_name'].nunique())
print('Total successful artists showing up on Massive Dance Hits:', successful_artists_df.loc[successful_artists_df['playlist_name'] == 'Massive Dance Hits']['artist_name'].nunique())
print('Total successful artists showing up on The Indie List:', successful_artists_df.loc[successful_artists_df['playlist_name'] == 'The Indie List']['artist_name'].nunique())
print('Total successful artists showing up on New Music Friday:', successful_artists_df.loc[successful_artists_df['playlist_name'] == 'New Music Friday']['artist_name'].nunique())

In [ ]:
successful_artists_frequency_playlists = successful_artists_df.groupby('playlist_name')['artist_name'].nunique().sort_values(ascending=False).reset_index(name='successful_artists')[:15]
#some_df[:20]

#successful_artists_frequency_playlists_chart = successful_artists_frequency_playlists['playlist_name'].value_counts()[:10].sort_values(ascending=True)

objects = successful_artists_frequency_playlists.playlist_name
y_pos = np.arange(len(objects))
performance = successful_artists_frequency_playlists.successful_artists
plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
plt.yticks(y_pos, objects)
plt.xlabel('Number of Unique Successful Artists')
plt.title('Playlists which contain the highest number of distinct successful artists')
plt.show()

In [ ]:
# Create the Cluser Map 
def plot_cluster(t,figsize=None):
    cg = sns.clustermap(t,figsize=(10, 8), cmap="mako", vmin=1)
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    return cg

In [ ]:
# Create a DataFrame with the playlists and artists by number of streams
playlists_filtered = data.stream_source_uri.value_counts().head(20).keys().tolist()
artists_filtered = data.artist_name.value_counts().head(20).keys().tolist()
df = data[:]
df['playlist_name'] = df.stream_source_uri.astype(str).str[-22:].map(playlist_mapper.set_index('id')['name'])
df = df.dropna(subset=['playlist_name','artist_name'])
df = df[(df.stream_source_uri.isin(playlists_filtered))&df.artist_name.isin(artists_filtered)]
df = df.groupby(['artist_name','playlist_name']).size().unstack().fillna(0)
plot_cluster(df)

<b>Distribution of streams per hour for the key playlists</b>

We observe that overall users tend to stream the key playlists mostly from 10 a.m. to 3 p.m. From 5 a.m. to 10 a.m the number of streams start to raise while from 3 p.m to 8 p.m the number of streams falls. During the late hours of the night there are not much streams. This is quite expected if we think of a normal person's day-to-day rutine.

We should also look at each key playlist's hourly trend. The Hot Hits UK playlist has the biggest number of streams somewhere between 3 p.m. and 8 p.m., while it sees a small decrease in the number of streams from 10 a.m to 3 p.m. Other than that, it follows the overall hourly trend discussed before. The Massive Dance Hits playlist seems to count the biggest number of streams at 3 p.m. and it generally follows the overall hourly trend discussed before. The New Music Friday playlist seems to also count the biggest number of streams at 3 p.m. We also observe an increase in the number of streams after 8 p.m. which does not happen for the other key playlists. Other than that, it tends to follow the overall hourly trend discussed before, but it has (most) signifficant ups and downs in number of streams. The Indie list playlists seems to count the biggest number of streams at 10 a.m. Other than that, it follows the overall hourly trend discussed before.

In [ ]:
# Bianca 
# Next, we want to analyse the distribution of streams per hour for the key playlists 

# 4 key Playlists 
key_playlists = playlist_mapper[playlist_mapper["name"].isin(["Hot Hits UK", "Massive Dance Hits", "The Indie List", "New Music Friday"])]
data_key_playlists = data[data.stream_source_uri.astype(str).str[-22:].isin(key_playlists.id)]

# Normalize to identify hourly trends 
key_playlists_streams_hour = data_key_playlists.groupby(['playlist_name','hour']).size().unstack().fillna(0).transpose()
key_playlists_streams_hour.plot(kind='line',figsize=(12,6))

# Visulaize the distribution of streams per hour (Normalized)
key_playlists_streams_hour_norm = pd.DataFrame(scale(key_playlists_streams_hour))
key_playlists_streams_hour_norm.columns = key_playlists_streams_hour.columns
key_playlists_streams_hour_norm.plot(title='Distribution Of Streams Per Hour (Normalized)',kind='line',figsize=(12,6))

By <b>removing the entires containing successful artists in the Hot Hits UK</b>, we can see that the number of such unique artists does not drop. A possible explaination could be that Hot Hits UK selects the most popular artists who are frequent in other playlists. Therefore, HHUK would not be the most appropriate benchmark to compare successful artists against. By looking at this playlist, we might lose focus from others, which might be more associated with the actual success level of an artist. 

Therefore, moving forward, we will disregard the strems taking place on Hot Hits UK.

In [ ]:
successful_artists_df_no_HHUK = successful_artists_df.loc[successful_artists_df.playlist_name != 'Hot Hits UK']
successful_artists_df_no_HHUK.artist_name.nunique()

In [ ]:
filter_playlists = successful_artists_df_no_HHUK.stream_source_uri.value_counts().head(20).keys().tolist()
filter_artists = successful_artists_df_no_HHUK.artist_name.value_counts().head(20).keys().tolist()
t = successful_artists_df_no_HHUK[:]
t['playlist_name'] = t.stream_source_uri.astype(str).str[-22:].map(playlist_mapper.set_index('id')['name'])
t = t.dropna(subset=['playlist_name','artist_name'])
t = t[(t.stream_source_uri.isin(filter_playlists))&t.artist_name.isin(filter_artists)]
t = t.groupby(['artist_name','playlist_name']).size().unstack().fillna(0)

plot_cluster(t)


### Artists Features
Because in the above analysis, we learned that dropping Hot Hits UK streams would actually allow better visibility over artists' presence on other playlists, we will have to remove these entires in the entire dataset.

We can look at an artist's success at multiple levels, vertically and horizontally.hence either to the actual number of streams and users it reached to, or to the playlists it reached or their history with Spotify. 

The 3 features suggested by Alaistar:
 - Stream count
 - Total Number of users
 - Passion Score

Besides the 3 features suggested by Alaistar, we could also look into: 
- Rank based on all time streams --> AVERAGE STREAMING PER ARTISTS
- Average it over few months (12)? 
- First time a track got played? - until it reached success?
- Podcasts? - types/gendres?
- How often a particular song is playes and shared? ~ nice to have but no data

https://community.spotify.com/t5/Content-Questions/What-does-the-Popular-tracks-on-an-artist-page-mean-How-is-it/td-p/4405230

Artists popularity is ranked by Spotify in several ways: 
- how often songs are played, therefore number of streams per song

In [ ]:
'''Because in the above analysis, we learned that dropping Hot Hits UK streams would actually allow better 
visibility over artists' presence on other playlists, we will have to remove these entires in the entire dataset.'''
df = data.loc[data.playlist_name != 'Hot Hits UK']

# 1. TOTAL NUMBER OF STREAMS PER ARTIST
artists_df = df['artist_name'].value_counts().to_frame().reset_index()
artists_df = artists_df.rename(columns={"index" : "artist_name",
                                         "artist_name" : "ar_stream_count"})
# 2. TOTAL NUMBER OF USERS PER ARTIST
df_artist_users = df.groupby('artist_name')['customer_id'].nunique().sort_values(ascending=False).reset_index(name='ar_users')

# Merging the two dataframes related to streams and users into one
artists_df = pd.merge(artists_df, df_artist_users, on='artist_name', how='outer')

# 3. PASSION SCORE
artists_df["ar_passion_score"]= artists_df['ar_stream_count']/artists_df['ar_users']
print("Range of users' passion score:", artists_df["ar_passion_score"].min(), artists_df["ar_passion_score"].max())

#==========NEW================
# 4. AVERAGE TRACK STREAMING PER ARTISTS --> on average, how frequent songs are streamed, by looking at the time stamp, rather than the user id
avg_streamed_tracks_per_artist = df.groupby(['artist_name', 'track_name'])['log_time'].nunique().groupby(['artist_name']).mean().sort_values(ascending = False).to_frame().reset_index().rename(columns={"log_time" : "ar_avg_tracks_streaming"})

# Merging the two dataframes related to streams and users into one
artists_df = pd.merge(artists_df, avg_streamed_tracks_per_artist, on='artist_name', how='outer')
artists_df 

### Playlists Features
Features suggested by Alaistar:
 - Prior Playlist Stream Counts
 - Prior Playlist Unique Users (Reach)
 - Prior Playlist Passion Score

Other features:
- Average number of streams per playlist - looking at the unique time stemp per every track, then averaging them per playlist
- History of a playlist - first track that playlist got ever streamed

In [ ]:
# 1. TOTAL STREAMS PER PLAYLIST
def playlist_avg_stream_counts(df):
    stream_count_per_playlist_df = df['playlist_name'].value_counts().to_frame().reset_index()
    return stream_count_per_playlist_df.rename(columns={"index" : "playlist_name", 
                                                        "playlist_name" : "pl_stream_count"})

In [ ]:
# 2. UNIQUE USERS PER PLAYLISTS
def playlist_avg_number_of_users(df):
    temp_df = df.groupby('playlist_name')['customer_id'].nunique().sort_values(ascending=False).reset_index(name='pl_users')
    return temp_df[['playlist_name', 'pl_users']]

In [ ]:

# Merging the two dataframes related to playlists into one
playlists_df = pd.merge(playlists_df, users_per_playlist, on='playlist_name', how='outer')

# 3. PLAYLISTS PASSIONSCORE
playlists_df["pl_passion_score"]= playlists_df['pl_stream_count']/playlists_df['pl_users']
print("Range of playlists' passion score:", playlists_df["pl_passion_score"].min(), playlists_df["pl_passion_score"].max())

#==========NEW==========
# 4. AVERAGE STREAMING PER PLAYLIST --> on average, how often songs are streamed, by looking at the time stamp, rather than the user id
avg_streamed_tracks_per_playlist = df.groupby(['playlist_name', 'track_name'])['log_time'].nunique().groupby(['playlist_name']).mean().sort_values(ascending = False).to_frame().reset_index().rename(columns={"log_time" : "avg_tracks_streaming"})

# Merging the two dataframes related to streams and users into one
playlists_df = pd.merge(playlists_df, avg_streamed_tracks_per_playlist, on='playlist_name', how='outer')
playlists_df 

<b> History of a playlist</b>

When evaluating playlists, we should also take into account their history within spotify. Ideally the older they are, the more reliability and consistency they would bring. These values could help us in creating better criteria for playlists assessment, which would subsequently help us better understand the level of success certain artists have. 
Because of the limited dataset, we will assess the history of a playlist based on the first track streamed.

In [ ]:
# First stream in every playlist
playlists_name = playlist_mapper.name.unique()
playlists_df_min_year = df.loc[df.playlist_name.isin(playlists_name)].groupby(['playlist_name'])['year'].min()
playlists_df_min_year = playlists_df_min_year.to_frame().reset_index().rename(columns={"year": "min_year"})

# Code for max
# playlists_df_max_year = df.loc[df.playlist_name.isin(playlists_name)].groupby(['playlist_name'])['year'].min()
# playlists_df_max_year = playlists_df_max_year.to_frame().reset_index().rename(columns={"year": "max_year"})
# playlists_years = pd.merge(playlists_df_min_year, playlists_df_max_year, on='playlist_name', how='outer')

#Create categorical data
playlists_years = pd.get_dummies(playlists_df_min_year, columns=['min_year'], drop_first=True)

#MERGING THE DF ABOVE
playlists_df = pd.merge(playlists_df, playlists_years, on='playlist_name', how='outer')
playlists_df 

### Additional Features
1. Streaming of successful artists across different playlists
2. Top playlists a particular artist was streamed the most in, different than the 4 playlists provided above


In [ ]:
def playlists_years_matrix(artists_name):
    specific_artist_df = df.loc[df['artist_name']== artists_name]
    pop_playlists = specific_artist_df['playlist_name'].value_counts()[:20].sort_values(ascending=False)
    temp_dff = specific_artist_df.loc[specific_artist_df['playlist_name'].isin(pop_playlists.index)].groupby(['date','playlist_name']).size().unstack().fillna(0)
    temp_dff = temp_dff.loc[:, (temp_dff != 0).any(axis=0)]
    temp_dff = temp_dff.reindex(temp_dff.mean().sort_values(ascending = False).index, axis=1)
    fig, ax = plt.subplots(figsize=(10,5))         # Sample figsize in inches
    plot_title = 'Playlists Outreach for ' + artists_name
    sns.heatmap(temp_dff.T, cmap = 'coolwarm', linewidths=.5, ax=ax).set_title(plot_title)

def show_top_playlists(artist_name):
    df_only_with_artist = df.loc[df['artist_name']== artist_name]
    artist_in_top_playlists = df_only_with_artist.loc[df_only_with_artist['successful']==False]['playlist_name'].value_counts()[:10].sort_values(ascending=True)
    fig = plt.figure()
    objects = artist_in_top_playlists.keys()
    y_pos = np.arange(len(objects))
    performance = artist_in_top_playlists.values
    plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
    plt.yticks(y_pos, objects)
    plt.xlabel('Number of Occurrences')
    plot_title = 'Key Playlists Different than "The 4" for ' + artist_name
    plt.title(plot_title)
    plt.show()

#For Testing   
# playlists_years_matrix('dua lipa')
# show_top_playlists('dua lipa')

In [ ]:
# # To get a better undertansing, 10 random artists were picked to be looked more closely into.
# #DRILL DOWN INTO 10 RANDOM SUCCESSFUL ARTISTS 
# import random
# from random import randint
# x = random.sample(range(0, 82), 10)
# pop_artists = df.loc[data['success'] == 1]['artist_name'].unique()
# random_successful_artists = pop_artists[x]

# for artist in random_successful_artists:
#     playlists_years_matrix(artist)
    
# for artist in random_successful_artists:
#     show_top_playlists(artist)

When looking at the above 10 randomly selected successful artists, we can notice that some artists become highly popular on various playlists, e.g. "Eurovision Related", "Every UK Number 1" or geners related e.g. pop, R&B. Therefore, the hypothesys is that playlists different than the 4 selected, can also be correlated to the likelyhood of an artist coming up on one of the 4 playlists; therefore, being popular on these playlists could be associated with "success".

We can also notive that out of the 4 playlsts associated with artists success ("Hot Hits UK", "Massive Dance Hits", "The Indie List", "New Music Friday") are not the playlists where they are also the most streamed. When looking at top 20 playlists for 10 random successful artists, only 2 of them show up.

In [ ]:
#AGGREGATE THE RESULTS OF ALL SUCCESSFUL ARTISTS
successful_artist_df = df.loc[df['artist_name'].isin(successful_artists)]
pop_playlists = successful_artist_df['playlist_name'].value_counts()[:20].sort_values(ascending=False)
temp_dff = successful_artist_df.loc[successful_artist_df['playlist_name'].isin(pop_playlists.index)].groupby(['date','playlist_name']).size().unstack().fillna(0)
temp_dff = temp_dff.loc[:, (temp_dff != 0).any(axis=0)]
temp_dff = temp_dff.reindex(temp_dff.mean().sort_values(ascending = False).index, axis=1)
fig, ax = plt.subplots(figsize=(10,5))         
plot_title = 'Top Playlists Different than the Top 4' 
sns.heatmap(temp_dff.T, cmap = 'coolwarm', linewidths=.5, ax=ax).set_title(plot_title)


artist_in_top_playlists = df.loc[df['successful']==False]['playlist_name'].value_counts()[:10].sort_values(ascending=True)
fig = plt.figure()
objects = artist_in_top_playlists.keys()
y_pos = np.arange(len(objects))
performance = artist_in_top_playlists.values
plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
plt.yticks(y_pos, objects)
plt.xlabel('Number of Occurrences')
plot_title = 'Key Playlists Different than "The 4" '
plt.title(plot_title)
plt.show()

Based on the matrix, we can see that few playlists actually have a consistent high number of streams in general. Most of them present some level of seasonality, for example, "Summer Hits" had the highest number of streams in the summer of 2016. Therefore, we should look at them also across years.

In [ ]:
print('Looking at most popular playlists over years')
def plot_top_playlists_by_year(year):
    artist_in_top_playlists = df.loc[df['year']==year]
    artist_in_top_playlists = artist_in_top_playlists.loc[artist_in_top_playlists['successful']==False]['playlist_name'].value_counts()[:10].sort_values(ascending=True)
    fig = plt.figure()
    objects = artist_in_top_playlists.keys()
    y_pos = np.arange(len(objects))
    performance = artist_in_top_playlists.values
    plt.barh(y_pos, performance, align='center', alpha=0.5, color = "#1aa64b")
    plt.yticks(y_pos, objects)
    plt.xlabel('Number of Occurrences')
    plot_title = 'Key Playlists Different than "The 4" in ' + str(year)
    plt.title(plot_title)
    plt.show()
    
plot_top_playlists_by_year(2015)
plot_top_playlists_by_year(2016)
plot_top_playlists_by_year(2017)

Based on these findings, we can observe there are several other playlists which contain a high number of streams of the successful artists and are fairly consistent over time. Surprisingly, but the other 3 playlists <b>"Massive Dance Hits", "The Indie List", "New Music Friday" </b> are not among them. Disregarding Hot Hits UK, we can notice that successful artists are the most streamed on these playlists: <b>'Today's Top Hits', 'Topsify UK Top 40', 'Freshness: Hot House Music', 'The Pop List'</b>. Therefore, similar to the top 4 "successful" playlists, we can label artists by whether they appeared on any of these playlists.

To avoid any potential bias on those playlsts, we can select the following 4 playlists and keep track of those in a similar fashion: <b>'New Music Monday UK', 'Happy Hits!', 'Summer Hits', 'Top Tracks in the United Kingdom'</b>.

In [ ]:
top_other_playlists_1 = ['Today\'s Top Hits', 'Topsify UK Top 40', 'Freshness: Hot House Music', 'The Pop List']
print("2nd Order Successful Playlists")
print(top_other_playlists_1)
print()

top_other_playlists_2 = ['New Music Monday UK', 'Happy Hits!', 'Summer Hits', 'Top Tracks in the United Kingdom']
print("3rd Order Successful Playlists")
print(top_other_playlists_2)

In [ ]:
def get_successful_artists_1(all_artists):
    uccessful_playlists_1 = all_artists[all_artists['playlist_name'].isin(top_other_playlists_1)]
    return uccessful_playlists_1['artist_name'].unique()

def get_successful_artists_2(all_artists):
    successful_playlists_2 = all_artists[all_artists['playlist_name'].isin(top_other_playlists_2)]
    return successful_playlists_2['artist_name'].unique()

successful_artists_1 = get_successful_artists_1(df)
successful_artists_2 = get_successful_artists_2(df)

# Create column with dependent variable "success", which takes value of 1if the artist is in the successful list
df['success_1'] = np.where(df['artist_name'].isin(successful_artists_1), 1, 0)
df['success_2'] = np.where(df['artist_name'].isin(successful_artists_2), 1, 0)

In [ ]:
temp_successful_artists_df = df.loc[df.success == 1]

temp_df_1 = temp_successful_artists_df.loc[temp_successful_artists_df['artist_name'].isin(successful_artists_1)]
print('Out of 82 successful artists, {} showed up on top_other_playlists_1.'.format(temp_df_1['artist_name'].nunique()))

temp_df_2 = temp_successful_artists_df.loc[temp_successful_artists_df['artist_name'].isin(successful_artists_2)]
print('Out of 82 successful artists, {} showed up on top_other_playlists_2:'.format(temp_df_2['artist_name'].nunique()))

### PLOT 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 10), dpi=80)

labels = ['Yes', 'No']
colors = ["#1aa64b","#363837"]

var1 = temp_df_1['artist_name'].nunique()
sizes1 = [var1, 82-var1]
ax1.set_title('Successful artists who showed up on top_other_playlists_1')
ax1.pie(sizes1,  explode=(0.04,0), labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, pctdistance=1.2,labeldistance=1.4)

var2 = temp_df_2['artist_name'].nunique()
sizes2 = [var2, 82-var2]
ax2.set_title('Successful artists who showed up on top_other_playlists_2')
ax2.pie(sizes2,  explode=(0.04,0), labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, pctdistance=1.2,labeldistance=1.4)

In [ ]:
# import gc
# gc.collect()

In [ ]:
# #trying to see the overlap
# temp_df_3 = temp_df_1.merge(temp_df_2, on='artist_name', how='left')
# temp_df_3
# fig = plt.figure(figsize=(8,5))

# var3 = temp_df_3['artist_name'].nunique()
# sizes3 = [var3, 82-var3]
# plt.title('Successful artists who showed up on both playlists')
# plt.pie(sizes3,  explode=(0.04,0), labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, pctdistance=1.2,labeldistance=1.4)

# plt.axis('equal')
# plt.show()

We can observe that majority of successful artists area streamed on these 2nd and 3rd order playlsits identified above. Therefore, we could consider looking at all 3 "success" variables and <b>compute our final dependent variable </b>.

We bring together artists and their top 20 playlists, along with the details of those 20 playlists (excluding HHUK) as identified above.

In [ ]:
df_ar_pl = df.groupby(['artist_name','playlist_name']).size()

# consider the top 20 playlists each artist has featured on
df_ar_pl = df_ar_pl.groupby(level=0, group_keys=False).nlargest(20)

df_ar_p1_reset = df_ar_pl.to_frame().reset_index()
df_ar_p1_reset.rename(columns={0:'count'}, inplace=True)
#df_ar_p1_reset

#======================================================================================
df_ar_p1_merge = df_ar_p1_reset.merge(playlists_df, on='playlist_name', how='inner')

df_ar_p1_merge = df_ar_p1_merge.sort_values(by=['artist_name', 'count'], ascending=[1,0]) # ascending doesn't do anything 
#df_ar_p1_merge

# add a column of the sum of playlist for each artist
artist_sum = df_ar_p1_merge.groupby(['artist_name']).agg({'count':sum})

df_ar_p1_merge = df_ar_p1_merge.merge(artist_sum,on='artist_name', how='inner', suffixes=('','_sum'))
#df_ar_p1_merge

#======================================================================================
df_ar_p1_merge['weight_score'] = df_ar_p1_merge['count'] * df_ar_p1_merge['pl_passion_score'] / df_ar_p1_merge['count_sum']

pl_avg = df_ar_p1_merge.groupby('artist_name').agg({'weight_score':sum})

# drop old weight score, to avoid duplicate.
df_ar_p1_merge.drop('weight_score', axis=1, inplace=True)

df_ar_p1_merge = df_ar_p1_merge.merge(pl_avg, on='artist_name', how='inner')
#df_ar_p1_merge

#======================================================================================
df_wght = df_ar_p1_merge[['artist_name','weight_score']]

df_wght = df_wght.drop_duplicates()

df_wght


In [ ]:
#==========Adding this variable to the artists dataframe =========
artists_df = artists_df.merge(df_wght, on='artist_name', how='left')
artists_df

<b>Playlist features</b>

We decide to create a dataframe that contains streamcount, number of users and the passion score of each playlist and then calculate the weighted average of top 20 playlists' passion score for each artist. Thus, we calculate the streamcount of each playlist and create the dataframe 'playlists_df'. Then we find out the number of users of each playlist and add it to the dataframe 'playlists_df'. Similarly, we calculate the passion score of each playlist by dividing streamcount by number of users and add it to the dataframe 'playlists_df'. For calculating the weighted average of top 20 playlists' passion scores for each artist, we start by finding the top 20 stream count of playlists for each artists. Then we merge it with the relevant playlist passion score and calculate the sum of the different playlist's stream count for each artist. Here we calculate the weighted average for each playlist of each artist. For example, an artist might occur in many playlists, so the stream count and the passion score of this artist in each playlist is different. We sum the total stream count for each artist from all the playlists they occur and then use the weighted average method to see the score of one artist in a particular playlist. 

The formula is: Stream count * playlist passion score / the sum of the stream count of one artist in different playlists.

Here we add the weighted average score to the dataframe. After that, we create a dataframe including the total weighted score of each artist and merge this into the dataframe 'df_artist'. This dataframe contains all the useful features variables.

## User-base features
- Gender Percentage Breakdown
- Age vector quantization

In [ ]:
#Proof that there are missing values
df.gender.unique()

In [ ]:
# Gender breakdown
df_gender = df.copy()

def gender_percentage(df_gen, gender='male'):
    if len(df_gen)!=0:
        # count the number of na
        num_na = df_gen.gender.isna().sum()
        
        # assume half of nan gender as 'male' 
        df_gen = df_gen.drop_duplicates(subset=['customer_id'])
        
        perc = (df_gen[df_gen.gender==gender].shape[0] + num_na/2)/len(df_gen)
        
        return round(perc, 4)
    else:
        return 0
    
gender_per = df_gender.groupby('artist_name').apply(gender_percentage, gender='male')
gender_per = gender_per.to_frame().reset_index()
gender_per = gender_per.rename(columns={0: "male_percentage"})
gender_per

In [ ]:
temp_artists_df = artists_df.merge(gender_per, on='artist_name', how='left')
temp_artists_df

In [ ]:
# Age breakdown

# groups of age
group_ages = [0,18,25,50,120]
group_names = ['Teen','Youth','Adult','Senior']

def age_group_per(df_new):
    df_new = df_new.dropna(subset=['birth_year'])
    
    df_new['age'] = df_new.year - df_new.birth_year
    
    df_new = df_new`.drop_duplicates(subset=['customer_id'])
    
    categories = pd.cut(df_new['age'], group_ages, labels=group_names)
    
    cate_per = {y:x/categories.value_counts().sum() for x,y in zip(categories.value_counts(), categories.value_counts().keys())}

    return pd.DataFrame([cate_per])

age_per = df.groupby('artist_name').apply(age_group_per)

# print(age_per)

# avoid collinearity, abandon one group
artists_df[['Youth','Adult','Senior']] = age_per.reset_index()[['Youth','Adult','Senior']]

artists_df